# Data Preprocessing Pipeline for Noisy Dataset

This notebook preprocesses the dataset with careful handling including:
- Leaving `S.No` column as is without scaling
- Filling missing Certificates and skill columns
- Encoding categorical columns except target and S.No
- Scaling numeric features except S.No
- Train-test splitting
- Removing outliers from Age column using z-score
- Saving intermediate and final outputs

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import joblib
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Load the dataset
df = pd.read_csv('noisy_dataset_v3.csv')
print("Dataset loaded successfully!")
print(f"Shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
df.head()

Dataset loaded successfully!
Shape: (1000, 12)

Columns: ['S.NO', 'Degree', 'Domain', 'Skill1', 'Skill2', 'Skill3', 'Certification', 'JobRole', 'Age', 'Location', 'Education_Level', 'University_Tier']


,S.NO,Degree,Domain,Skill1,Skill2,Skill3,Certification,JobRole,Age,Location,Education_Level,University_Tier
0,1,B.Com,Finance,PowerBI,Financial Modeling,Leadership,CFA,Financial Analyst,22.0,Australia,Bachelor,Tier 2
1,2,B.Des,Design,Figma,UI/UX,Communication,NaN,UI/UX Designer,25.0,USA,Master,Tier 3
2,3,MBA,Finance,Financial Modeling,Data Analysis,Excel,NaN,Financial Analyst,22.0,Australia,Bachelor,Tier 2
3,4,MBA,Data Science,PowerBI,Data Analysis,Research,NaN,Data Analyst,28.0,India,Master,Tier 3
4,5,B.Com,Marketing,Excel,Marketing,Communication,NaN,Marketing Specialist,26.0,Australia,Bachelor,Tier 3


In [3]:
# Initial data inspection
print("Data Info:")
print(df.info())
print("\nMissing Values:")
print(df.isnull().sum())
print(f"\nDuplicate rows: {df.duplicated().sum()}")
print("\nBasic Statistics:")
print(df.describe())

Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   S.NO             1000 non-null   int64  
 1   Degree           1000 non-null   object 
 2   Domain           994 non-null    object 
 3   Skill1           997 non-null    object 
 4   Skill2           997 non-null    object 
 5   Skill3           997 non-null    object 
 6   Certification    265 non-null    object 
 7   JobRole          997 non-null    object 
 8   Age              982 non-null    float64
 9   Location         994 non-null    object 
 10  Education_Level  1000 non-null   object 
 11  University_Tier  1000 non-null   object 
dtypes: float64(1), int64(1), object(10)
memory usage: 93.9+ KB
None

Missing Values:
S.NO                 0
Degree               0
Domain               6
Skill1               3
Skill2               3
Skill3               3
Certification  

In [4]:
# Handle missing values in Certification column
if 'Certification' in df.columns:
    df['Certification'] = df['Certification'].fillna('None')
elif 'Certificates' in df.columns:
    df['Certificates'] = df['Certificates'].fillna('None')
print("Missing Certification values filled with 'None'")
print(f"\nMissing values after Certification handling:")
print(df.isnull().sum())

Missing Certification values filled with 'None'

Missing values after Certification handling:
S.NO                0
Degree              0
Domain              6
Skill1              3
Skill2              3
Skill3              3
Certification       0
JobRole             3
Age                18
Location            6
Education_Level     0
University_Tier     0
dtype: int64


In [5]:
# Handle missing values in skill columns based on JobRole mode
skill_columns = ['Skill1', 'Skill2', 'Skill3']

for skill_col in skill_columns:
    if skill_col in df.columns:
        print(f"Processing {skill_col}...")

        def fill_skill_by_jobrole(row):
            if pd.isnull(row[skill_col]) or row[skill_col] == '':
                jobrole = row['JobRole']
                mode_val = df[df['JobRole'] == jobrole][skill_col].mode()
                if len(mode_val) > 0:
                    return mode_val[0]
                else:
                    overall_mode = df[skill_col].mode()
                    if len(overall_mode) > 0:
                        return overall_mode[0]
                    else:
                        return 'Skill_Missing'
            else:
                return row[skill_col]

        df[skill_col] = df.apply(fill_skill_by_jobrole, axis=1)
        print(f"{skill_col} missing values handled based on JobRole mode")

print(f"\nMissing values after skill handling:")
print(df.isnull().sum())

Processing Skill1...
Skill1 missing values handled based on JobRole mode
Processing Skill2...
Skill2 missing values handled based on JobRole mode
Processing Skill3...
Skill3 missing values handled based on JobRole mode

Missing values after skill handling:
S.NO                0
Degree              0
Domain              6
Skill1              0
Skill2              0
Skill3              0
Certification       0
JobRole             3
Age                18
Location            6
Education_Level     0
University_Tier     0
dtype: int64


In [6]:
# Handle remaining missing values
for col in df.columns:
    if df[col].isnull().sum() > 0:
        if df[col].dtype == 'object':
            mode_val = df[col].mode()
            if len(mode_val) > 0:
                df[col].fillna(mode_val[0], inplace=True)
            else:
                df[col].fillna('Unknown', inplace=True)
        else:
            df[col].fillna(df[col].median(), inplace=True)
        print(f"Remaining missing values in {col} handled")

print(f"\nFinal missing values check:")
print(df.isnull().sum())

Remaining missing values in Domain handled
Remaining missing values in JobRole handled
Remaining missing values in Age handled
Remaining missing values in Location handled

Final missing values check:
S.NO               0
Degree             0
Domain             0
Skill1             0
Skill2             0
Skill3             0
Certification      0
JobRole            0
Age                0
Location           0
Education_Level    0
University_Tier    0
dtype: int64


In [7]:
# Remove duplicates
initial_shape = df.shape[0]
df = df.drop_duplicates()
final_shape = df.shape[0]
print(f"Removed {initial_shape - final_shape} duplicate rows")
print(f"Final dataset shape: {df.shape}")

Removed 0 duplicate rows
Final dataset shape: (1000, 12)


In [8]:
# Remove outliers using Z-score method
age_mean = df['Age'].mean()
age_std = df['Age'].std()
df = df[(np.abs((df['Age'] - age_mean) / age_std)) < 3]
print("\nData info after removing Age outliers:")
print(df.info())
print("\nMissing values after outlier removal:")
print(df.isnull().sum())


Data info after removing Age outliers:
<class 'pandas.core.frame.DataFrame'>
Index: 990 entries, 0 to 999
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   S.NO             990 non-null    int64  
 1   Degree           990 non-null    object 
 2   Domain           990 non-null    object 
 3   Skill1           990 non-null    object 
 4   Skill2           990 non-null    object 
 5   Skill3           990 non-null    object 
 6   Certification    990 non-null    object 
 7   JobRole          990 non-null    object 
 8   Age              990 non-null    float64
 9   Location         990 non-null    object 
 10  Education_Level  990 non-null    object 
 11  University_Tier  990 non-null    object 
dtypes: float64(1), int64(1), object(10)
memory usage: 100.5+ KB
None

Missing values after outlier removal:
S.NO               0
Degree             0
Domain             0
Skill1             0
Skill2             0
Sk

In [9]:
# Save the cleaned dataset (before encoding)
df.to_csv('cleaned_dataset_before_encoding.csv', index=False)
print("Cleaned dataset (before encoding) saved as 'cleaned_dataset_before_encoding.csv'")

print("\nSample of cleaned data:")
df.head()

Cleaned dataset (before encoding) saved as 'cleaned_dataset_before_encoding.csv'

Sample of cleaned data:


,S.NO,Degree,Domain,Skill1,Skill2,Skill3,Certification,JobRole,Age,Location,Education_Level,University_Tier
0,1,B.Com,Finance,PowerBI,Financial Modeling,Leadership,CFA,Financial Analyst,22.0,Australia,Bachelor,Tier 2
1,2,B.Des,Design,Figma,UI/UX,Communication,None,UI/UX Designer,25.0,USA,Master,Tier 3
2,3,MBA,Finance,Financial Modeling,Data Analysis,Excel,None,Financial Analyst,22.0,Australia,Bachelor,Tier 2
3,4,MBA,Data Science,PowerBI,Data Analysis,Research,None,Data Analyst,28.0,India,Master,Tier 3
4,5,B.Com,Marketing,Excel,Marketing,Communication,None,Marketing Specialist,26.0,Australia,Bachelor,Tier 3


In [10]:
# Encode categorical variables (except target JobRole and exclude S.No)
categorical_cols = df.select_dtypes(include='object').columns.difference(['JobRole'])
if 'S.No' in categorical_cols:
    categorical_cols = categorical_cols.drop('S.No')
label_encoders = {}

print(f"Encoding categorical columns: {categorical_cols.tolist()}")

for col in categorical_cols:
    le = LabelEncoder()
    df[col] = df[col].astype(str)  # Ensure string type
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le
    print(f"{col} encoded with {len(le.classes_)} unique values")

print("\nCategorical encoding completed!")

Encoding categorical columns: ['Certification', 'Degree', 'Domain', 'Education_Level', 'Location', 'Skill1', 'Skill2', 'Skill3', 'University_Tier']
Certification encoded with 8 unique values
Degree encoded with 12 unique values
Domain encoded with 17 unique values
Education_Level encoded with 4 unique values
Location encoded with 7 unique values
Skill1 encoded with 88 unique values
Skill2 encoded with 87 unique values
Skill3 encoded with 9 unique values
University_Tier encoded with 3 unique values

Categorical encoding completed!


In [11]:
# Scale numerical features excluding S.No
numerical_cols = df.select_dtypes(include=[np.number]).columns.difference(['S.No'])
print(f"Scaling numerical columns (excluding S.No): {numerical_cols.tolist()}")

scaler = StandardScaler()
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

print("Numerical scaling completed!")

Scaling numerical columns (excluding S.No): ['Age', 'Certification', 'Degree', 'Domain', 'Education_Level', 'Location', 'S.NO', 'Skill1', 'Skill2', 'Skill3', 'University_Tier']
Numerical scaling completed!


In [12]:
# Save the fully cleaned and encoded dataset
df.to_csv('fully_cleaned_encoded_dataset.csv', index=False)
print("Fully cleaned and encoded dataset saved as 'fully_cleaned_encoded_dataset.csv'")

print("\nSample of fully processed data:")
df.head()

Fully cleaned and encoded dataset saved as 'fully_cleaned_encoded_dataset.csv'

Sample of fully processed data:


,S.NO,Degree,Domain,Skill1,Skill2,Skill3,Certification,JobRole,Age,Location,Education_Level,University_Tier
0,-1.729648,-1.930138,0.134308,0.398432,-0.516371,-0.010231,-1.144320,Financial Analyst,-1.397077,-1.416715,-0.901787,-0.180353
1,-1.726174,-1.597587,-0.266189,-0.710144,1.924270,-1.568503,0.328862,UI/UX Designer,-0.751269,1.565314,0.929534,1.248046
2,-1.722701,1.062819,0.134308,-0.669085,-0.944553,-0.789367,0.328862,Financial Analyst,-1.397077,-1.416715,-0.901787,-0.180353
3,-1.719228,1.062819,-0.466437,0.398432,-0.944553,1.158473,0.328862,Data Analyst,-0.105460,0.074300,0.929534,1.248046
4,-1.715755,-1.930138,0.735053,-0.792260,-0.045370,-1.568503,0.328862,Marketing Specialist,-0.535999,-1.416715,-0.901787,1.248046


In [14]:
# Prepare features and target
df.set_index('S.NO', inplace=True)

X = df.drop('JobRole', axis=1)
y = df['JobRole']

# Encode target variable
target_encoder = LabelEncoder()
y_encoded = target_encoder.fit_transform(y.astype(str))

print(f"Features shape: {X.shape}")
print(f"Target shape: {y_encoded.shape}")
print(f"Unique job roles: {len(target_encoder.classes_)}")
print(f"Job roles: {target_encoder.classes_}")

Features shape: (990, 10)
Target shape: (990,)
Unique job roles: 24
Job roles: ['Biotech Researcher' 'Business Analyst' 'Business Development Manager'
 'Chemical Engineer' 'Civil Engineer' 'Cloud Engineer' 'Content Writer'
 'Data Analyst' 'Data Engineer' 'Data Scientist' 'Electrical Engineer'
 'Financial Analyst' 'HR Manager' 'Machine Learning Engineer'
 'Marketing Specialist' 'Mechanical Engineer' 'Network Engineer'
 'Operations Manager' 'Project Manager' 'Sales Manager' 'Scrum Master'
 'Security Analyst' 'Supply Chain Manager' 'UI/UX Designer']


In [15]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

print(f"Training set: {X_train.shape}")
print(f"Testing set: {X_test.shape}")
print(f"Training target: {y_train.shape}")
print(f"Testing target: {y_test.shape}")

Training set: (792, 10)
Testing set: (198, 10)
Training target: (792,)
Testing target: (198,)


In [16]:
# Save processed train-test datasets
X_train.to_csv('X_train_preprocessed.csv', index=False)
X_test.to_csv('X_test_preprocessed.csv', index=False)
pd.DataFrame(y_train, columns=['JobRole_encoded']).to_csv('y_train_preprocessed.csv', index=False)
pd.DataFrame(y_test, columns=['JobRole_encoded']).to_csv('y_test_preprocessed.csv', index=False)

# Save encoders for future use
joblib.dump(label_encoders, 'categorical_label_encoders.pkl')
joblib.dump(scaler, 'numerical_scaler.pkl')
joblib.dump(target_encoder, 'target_encoder.pkl')

print("All preprocessed files saved successfully!")

print("\nFiles created:")
print("- cleaned_dataset_before_encoding.csv")
print("- fully_cleaned_encoded_dataset.csv")
print("- X_train_preprocessed.csv")
print("- X_test_preprocessed.csv")
print("- y_train_preprocessed.csv")
print("- y_test_preprocessed.csv")
print("- categorical_label_encoders.pkl")
print("- numerical_scaler.pkl")
print("- target_encoder.pkl")

All preprocessed files saved successfully!

Files created:
- cleaned_dataset_before_encoding.csv
- fully_cleaned_encoded_dataset.csv
- X_train_preprocessed.csv
- X_test_preprocessed.csv
- y_train_preprocessed.csv
- y_test_preprocessed.csv
- categorical_label_encoders.pkl
- numerical_scaler.pkl
- target_encoder.pkl


In [17]:
# Final summary
print("=== PREPROCESSING SUMMARY ===")
print(f"Original dataset shape: {df.shape}")
print(f"Features: {X.shape[1]}")
print(f"Samples: {X.shape[0]}")
print(f"Classes: {len(target_encoder.classes_)}")
print("\n=== PREPROCESSING COMPLETED SUCCESSFULLY! ===")

=== PREPROCESSING SUMMARY ===
Original dataset shape: (990, 11)
Features: 10
Samples: 990
Classes: 24

=== PREPROCESSING COMPLETED SUCCESSFULLY! ===


In [ ]:
df.to_csv('fully_preprocessed_dataset.csv', index=False)